# Sample Code BE565
#### Python script to simulate collective cell chemotaxis due to inhibited front contractility of a supracellular actomyosin ring present in the peripheral cells of a cell cluster. This script is part of a study by Adam Shellard, Andras Szabo, Xavier Trepat, and Roberto Mayor, published in Science, 2018. If you are using this code, please cite the original publication:
#### Adam Shellard, András Szabó, Xavier Trepat, Roberto Mayor. Supracellular contraction at the rear of neural crest cell groups drives collective chemotaxis. Science 362(6412): 339-343, 2018. DOI: 10.1126/science.aau3301 
#### This script is written by Andras Szabo (andras.szabo.81@gmail.com). 
 
#### Main dependencies used for running the script:
#### python 2.7.9 (note from maggie - I adjusted this for my python (version 3.5.3 | Anaconda Custom (64-bit) )
#### scipy 1.1.0  
#### numpy 1.15.3  
#### matplotlib 2.2.3      


In [11]:
## Dependencies
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay
from collections import defaultdict
import sys
print(sys.version)

3.5.3 |Anaconda custom (64-bit)| (default, May 15 2017, 10:43:23) [MSC v.1900 64 bit (AMD64)]


## Input Paramaters for the Model below

In [12]:
# Geometry of the simulation
global_initial_areas	= np.array([ [130,130], [270,270] ])	# initiate cells in these ranges: from (x,y) -> to (x,y)
global_Lmax		= np.array([400, 600])			# size of area from [0,0] to this position
global_N		= 60					# number of cells

# --------------------------------------------------- 
# General simulation parameters
global_Tmax		= 500					# maximum time of simulations
## NOTE: Maggie changed this from 10,000 to 1000 for times sake

global_Tout		= 20					# time-frames for output
global_dt		= 0.1					# time-step for Euler iterations
global_full_contraction = False					# set isotropic contraction, otherwise back half contracts
global_intercalate_until= global_Tmax				# allow neighbour exchange until this time
####--------------------------------------------------------------------------------------####

# ---------------------------------------------------
# Cell size and other lengths, radii
global_ranges 		= [6., 9., 9.2, 11.]			
# repulsion core, shortest adhesion distance, maximum adhesion force distance, furthest reach of the cell

# --------------------------------------------------- # Other cellular properties
global_speed		= 2					# target speed of cells
global_velDampening	= 0.1					# velocity dampening, or friction of movement (less than 1)
global_contraction_time	= 100					# minimum duration of contractions
global_relaxation_time	= 100					# minimum time between contractions

# --------------------------------------------------- #	Interaction strengths (negative = attraction)
global_I_rnd 		= 5					# random movement strength of cell
global_I_spp		= 5					# self-propusion strength of cell
global_I_rep 		= 10					# core cell repulsion strength
global_I_contact	= -0.5					# cell-cell adhesion strength
global_I_contraction	= -5					# actomyosin contraction strength

In [13]:

class Cell(object):
	type = 0	# cell type is 0 for peripheral cells and 1 for internal cells
	pos = np.array((2))
	newpos = np.array((2))
	vel = np.array((2))
	newvel = np.array((2))
	acc = np.array((2))
	
	pol = np.asarray([0.0, 0.0])
	pol_age = 0

	speed = 0
	velDampening = 0
	
	steric_radius = 0
	neutral_radius = 0
	adhesion_radius = 0
	reach_radius = 0
	
	steric_coeff = 0
	adhesion_coeff = 0
	speed_coeff = 0
	rnd_coeff = 0
	
	steric_gradient = 0
	adhesion_gradientA = 0
	adhesion_gradientB = 0

	actomyosin_contraction = False
	actomyosin_off_time = 0
	actomyosin_on_time = 0

	neighbor = []
	
	
	def __init__(self, pos, type, I, R, props, N):
		self.pos = np.asarray(pos)
		self.newpos = np.asarray(pos)
		vel = np.array([0,0])
		self.vel = np.asarray(vel)
		self.newvel = np.asarray(vel)
		self.type = type
		
		self.speed = props[0]
		self.velDampening = props[1]

		self.steric_coeff = I[0]
		self.adhesion_coeff = I[1]
		self.speed_coeff = I[2]
		self.rnd_coeff = I[3]
		
		self.reset_ranges(R)

	def reset_ranges(self, new_ranges):
		# Re-adjust the ranges of adhesion and repulsion interactions

		self.steric_radius, self.neutral_radius, self.adhesion_radius, self.reach_radius = \
			new_ranges[0], new_ranges[1], new_ranges[2], new_ranges[3]

		self.steric_gradient = self.steric_coeff / self.steric_radius
		self.adhesion_gradientA = self.adhesion_coeff / (self.adhesion_radius - self.neutral_radius)
		self.adhesion_gradientB = self.adhesion_coeff / (self.reach_radius - self.adhesion_radius)

			
	def actomyosin_contraction_on(self):
		ret = False

		if self.actomyosin_off_time >= global_relaxation_time:
			self.actomyosin_contraction = True
			ret = True
		return ret

	def actomyosin_contraction_off(self):
		ret = False

		if self.actomyosin_on_time >= global_contraction_time:
			self.actomyosin_contraction = False
			ret = True
		return ret


	def plot_colour(self):
		# return the colour to be used for plotting this cell
		
		if self.type == 0:
			col = [0.5, 0.5, 0.6]		# Default cell colour
			if self.actomyosin_contraction:
				col = [0.8, 0.75, 0.1]	# Contracting cell colour
		if self.type == 1:
			col = [0.2, 0.61, 0.35]
		return col
	
def unit_vector(v):
	# Return the unit vector of the 2D vector v
	length = np.linalg.norm(v)
	if length > 0:
		return (v / length)
	else:
		return 0 * v


def initiateCells(N, areas, I, R, props):
	
	# Initiate cells in a pre-set geometry

	cell = []
	type = 1
	Ntotal = np.sum(N)
	id = 0
	
	centre = np.asarray([(areas[1][0] + areas[0][0]) * 0.5, (areas[1][1] + areas[0][1]) * 0.5])
	radius = (areas[1][0] - areas[0][0]) * 0.5
	dx = areas[1][0] - areas[0][0]
	dy = areas[1][1] - areas[0][1]
	xmin = areas[0][0]
	ymin = areas[0][1]
	for counter in range(N):
		pos = np.asarray([0.0,0.0])
		too_close = True
		tries = 0 
		while (too_close):
			pos = np.asarray([random.random() *dx + xmin, random.random() *dy + ymin])
			if (np.linalg.norm(pos-centre) < radius) : 
				mindist=-1
				for i in range(id):
					dist = np.linalg.norm( pos - cell[i].pos )
					if mindist < 0 or mindist > dist:
						mindist = dist
				if mindist <0 or mindist > 2.0 * cell[i].steric_radius:
					too_close=False
				else:
					if tries > 500:
						print ("Too high density, failed to initialize")
						exit()
			tries += 1
			if tries > 500:
				print ("Too high density, failed to initialize")
				exit()
		c = Cell(pos, type, I, R, props, Ntotal)
		cell.append(c)
		id += 1

	return cell

			
def iterate(cell, dt, time):
	# Iterate the position of all cells 

	for id in range(len(cell)):

		#--- Components controlling cell displacement / movement ---#
		# ++ Interaction with neighbour cells (neighbours are determined through the update_cells function)
		f_interaction = np.asarray([0.0,0.0])
		for nid in cell[id].neighbor:
			if (id != nid):
				f, dist = interact(cell, id, nid)
				f_interaction += f

		# ++ Self-propulsion
		f_spp = cell[id].speed_coeff * (cell[id].speed - np.linalg.norm(cell[id].vel)) * unit_vector(cell[id].vel)
		
		# ++ Noise
		f_noise = cell[id].rnd_coeff * (np.asarray([random.random(), random.random()]) - 0.5)
		
		
		
		# Cell movement dynamics: 'friction' + interaction + spp + noise 
		cell[id].newvel = dt * ((1.0 - cell[id].velDampening) * cell[id].vel + f_interaction + f_spp + f_noise)
		cell[id].newpos = cell[id].pos + dt * cell[id].vel

		# Closed boundary conditions: cannot go into walls
		if cell[id].newpos[0] < 0:		# left 
			if cell[id].newvel[0] < 0:
				cell[id].newvel[0] = 0
		if cell[id].newpos[0] > global_Lmax[0]:	# right
			if cell[id].newvel[0] > 0:
				cell[id].newvel[0] = 0
		if cell[id].newpos[1] < 0: 		# top
			if cell[id].newvel[1] < 0:
				cell[id].newvel[1] = 0
		if cell[id].newpos[1] > global_Lmax[1]:	# bottom
			if cell[id].newvel[1] > 0:
				cell[id].newvel[1] = 0
		#----

		
	# Update cell positions
	for cellid in range(len(cell)):
		cell[cellid].pos = cell[cellid].newpos
		cell[cellid].vel = cell[cellid].newvel

	return cell
def interact(cell, cellid, ncellid):
	# Interaction of cell 'cellid' and cell 'ncellid'. 
	# Cell information is stored in the cell object
	# Return the effect (or "force") on cell a. 
	
	f = np.asarray([0.0,0.0])
	m = 0 
	dist = np.linalg.norm(cell[cellid].pos - cell[ncellid].pos)
	dir = unit_vector(cell[cellid].pos - cell[ncellid].pos)
	
	### The interaction "potential" ###

	if dist < (cell[cellid].reach_radius + cell[ncellid].reach_radius):
		# Only interact if within range

		if dist < (cell[cellid].steric_radius + cell[ncellid].steric_radius):
			# repell:
			delta = cell[cellid].steric_radius + cell[ncellid].steric_radius - dist 
			m += cell[cellid].steric_gradient * delta

		elif dist < (cell[cellid].neutral_radius + cell[ncellid].neutral_radius):
			# cells are in the neutral region, no effect on movement
			m += 0

		elif dist < (cell[cellid].adhesion_radius + cell[ncellid].adhesion_radius):
			# attract
			delta = dist - (cell[cellid].neutral_radius + cell[ncellid].neutral_radius)
			m += cell[cellid].adhesion_gradientA * delta 

		elif dist < (cell[cellid].reach_radius + cell[ncellid].reach_radius):
			# attract
			delta = dist - (cell[cellid].adhesion_radius + cell[ncellid].adhesion_radius)
			m += cell[cellid].adhesion_gradientB * delta 

		# Special interaction between peripheral cells
		if cell[cellid].type == 0 and cell[ncellid].type == 0:

			# If both cells contract, add myosin force:
			if cell[cellid].actomyosin_contraction and cell[ncellid].actomyosin_contraction: 
				m += global_I_contraction	# Use a constant attraction force

	f = m * dir
	return f, dist
def set_peripheral_or_central_cell_type(cell):
	# Decide for each cell whether it is at the periphery (type=0) or internal to a cluster (type=1)

	# The angle between neighbouring connections above which the cell is considered to be at the periphery:
	threshold = np.pi * 0.75
	
	# 1. Determine neighbours
	if len(cell) > 2:
		# Use Voronoi / Delaunay tessellation to infer neighbours:
		pos = np.zeros((len(cell), 2))
		for id in range(len(cell)):		# Collect the positions of the cells in an array
			pos[id][0] = cell[id].pos[0]
			pos[id][1] = cell[id].pos[1]
		tessellation = Delaunay(pos)

		# Determine neighbours of all cells:
		neighbors = defaultdict(set)
		for simplex in tessellation.vertices:
			for idx in simplex:
				other = set(simplex)
				other.remove(idx)
				neighbors[idx] = neighbors[idx].union(other)

				# Only those cells count as neighbours, which are within reach:
				cell[idx].neighbor = []
				for nid in neighbors[idx]:
					if (np.linalg.norm( cell[idx].pos - cell[nid].pos) < \
					   (cell[idx].reach_radius + cell[nid].reach_radius)):
						# cells are within reach of each other
						cell[idx].neighbor.append(nid)
	else:
		# For two cells only, neighbours can be determined simply based on distance
		if len(cell) == 2:
			if np.linalg.norm(cell[0].pos - cell[1].pos) < cell[0].reach_radius + cell[1].reach_radius:
				cell[0].neighbor = [1]
				cell[1].neighbor = [0]
			else:
				cell[0].neighbor = []
				cell[1].neighbor = []
		else:
			cell[0].neighbor = []
			

	# 2. Based on the neighbours, determine if each cell is at the periphery (0) or centre (1):
	for id in range(len(cell)):
		nxs = []
		nys = []
		nids = []

		for nid in cell[id].neighbor:
			nxs.append(cell[nid].pos[0] - cell[id].pos[0])
			nys.append(cell[nid].pos[1] - cell[id].pos[1])
			nids.append(nid)
		
		nangles = np.sort(np.arctan2(nys, nxs))
		diffangles = []

		# detect if there is a gap between two consecutive angles that are
		# more than 'threshold' apart --> this will be considered as a free edge (type = 0)
		if len(nangles) > 3:
			type = 1
			for a in range(len(nangles)-1):
				diffangles.append(nangles[a+1] - nangles[a])
			diffangles.append(nangles[0] + 2.0*np.pi - nangles[len(nangles)-1])
			
			if np.max(diffangles) > threshold:
				type = 0
		else:
			type = 0
		cell[id].type = type
def update_cell_dynamics (cell, dt):
	# Advance any dynamics in the cells

	for id in range(len(cell)):
		if cell[id].actomyosin_contraction:
			cell[id].actomyosin_on_time += dt
			cell[id].actomyosin_off_time = 0
		else:
			cell[id].actomyosin_on_time   = 0
			cell[id].actomyosin_off_time += dt
def update_cells (cell, iteration, time, dt, counter):
	# Apply any manipluation to the cells here if needed
	# time is the number of iteration step rather than 'real time'
	
	
	# Decide the type of the cell: peripherial (0) or internal (1)
	if time <= global_intercalate_until :
		set_peripheral_or_central_cell_type(cell)
	update_cell_dynamics(cell, dt)
	counter += dt
	if counter > global_contraction_time:
		counter = -global_relaxation_time
	

	if (iteration % 10 and time > 100):
		pxs, pys = [], []
		for id in range(len(cell)):
			pxs.append(cell[id].pos[0])
			pys.append(cell[id].pos[1])
		cmx, cmy = np.mean(pxs), np.mean(pys)
		sizex = np.max(pxs) - np.min(pxs)
		sizey = np.max(pys) - np.min(pys)

		# Update actomyosin contaction properties:
		for id in range(len(cell)):
			if cell[id].type == 0:
				# only perturb the properties of the cells at the periphery
				
				# determine positional direction of the cell with respect to the centre of mass
				if global_full_contraction or time < 200:
					# uniform contractions before the initial period and in isotropic contraction 
					# simulations; set direction to back (along positive y-axis)
					direction = np.arctan2(1,0)
				else: 
					# switch on and off back cells together periodically:
					direction = np.arctan2(cmy - cell[id].pos[1], cmx - cell[id].pos[0])
				
				# contract at the back but not the front:
				if direction > np.pi * (-8.0/8.0) and direction < np.pi * (-0.0/8.0):
					# At the front, don't turn on
					cell[id].actomyosin_contraction = False
				else:
					# At the back, consider contracting
					if counter > 0:
						cell[id].actomyosin_contraction = True
					else:
						cell[id].actomyosin_contraction = False
	return counter
					
def output_state(cell, t, Lmax):
	# Output an image frame and print the coordinates to stdout

	# Plot:
	fig = plt.gcf()
	ax = plt.gca()
	ax.cla()
	# Uncomment to draw grid:
	#ax.grid(b=True, which='major')
	# Draw boundaries
	fig.gca().add_artist(plt.Rectangle((0,0),Lmax[0],Lmax[1],fill=False, color='0.75'))

	# Draw cells
	for i in range(len(cell)):
		col = cell[i].plot_colour()
		s = cell[i].steric_radius
		if s == 0:
			s=2
		cx=cell[i].pos[0]
		cy=cell[i].pos[1]
		fig.gca().add_artist(plt.Circle((cx,cy),s,color=col,fill=True))

	bufferx, buffery = 0.25*Lmax[0], 0.25*Lmax[1]
	if bufferx < 10:
		bufferx = 10
	if buffery < 10:
		buffery = 10
	if bufferx > 100:
		bufferx = 100
	if buffery > 100:
		buffery = 100
	# Add label
	fig.gca().add_artist(plt.Text(x=(Lmax[0]*0.75),y=(Lmax[1]+buffery*0.5), text=str(t), \
		horizontalalignment='right', size='16'))
	ax.set_xlim(-bufferx, Lmax[0]+bufferx)
	ax.set_ylim(-buffery, Lmax[1]+buffery)
	ax.set_aspect(1)
	ax.invert_yaxis()
	fig.savefig('config-'+"{0:05d}".format(int(t))+'.png', dpi=150, bbox_inches='tight')

	# Print coordinates
	for id in range(len(cell)):
		contraction = 0
		if cell[id].actomyosin_contraction:
			contraction = 1
		print (t, cell[id].type, id, cell[id].pos[0], cell[id].pos[1], contraction)
	print ("\n")





In [14]:
def main():
	cellsize = global_ranges[0]
	N = global_N
	Lmax = global_Lmax
	Tmax = global_Tmax
	t_out = global_Tout
	dt = global_dt
	counter = -1.0 * global_relaxation_time	
	
	# Read coefficients:
	interaction_coeffs = np.transpose(np.array([global_I_rep, global_I_contact, global_I_spp, \
		global_I_rnd]))
	spatial_ranges = np.transpose(np.array(global_ranges))
	props = np.transpose(np.array([global_speed, global_velDampening]))


	# Initiate cells:
	cell = initiateCells(N, global_initial_areas, interaction_coeffs, \
		spatial_ranges, props)

	# Save initial configuration and start simulation
	output_state (cell, 0.0, Lmax)
	for it in range(1,int(Tmax / dt)+1):
		t = it * dt		
		counter = update_cells(cell, it, t, dt, counter)
		iterate(cell, dt, t) 
		if (t % t_out) == 0 :
			output_state(cell, t, Lmax)
	
	

if __name__ == "__main__":
	main()

0.0 1 0 205.587098937 259.552986202 0
0.0 1 1 257.908923605 182.177282492 0
0.0 1 2 138.049968429 182.076017948 0
0.0 1 3 221.809681152 242.623545413 0
0.0 1 4 154.041787556 205.762620785 0
0.0 1 5 158.202949423 235.383688515 0
0.0 1 6 169.9736397 173.667020502 0
0.0 1 7 176.954881615 194.740008811 0
0.0 1 8 177.755447361 159.065124438 0
0.0 1 9 168.243126069 208.294502303 0
0.0 1 10 243.882597342 190.260673342 0
0.0 1 11 171.372371987 136.641852429 0
0.0 1 12 200.729991544 227.441825937 0
0.0 1 13 222.818578253 214.662613698 0
0.0 1 14 185.28280242 259.74455211 0
0.0 1 15 222.180678556 151.688664208 0
0.0 1 16 146.959299471 222.953792859 0
0.0 1 17 143.96345146 171.193213042 0
0.0 1 18 237.6059741 149.424944605 0
0.0 1 19 203.747144144 209.722888439 0
0.0 1 20 232.972415174 261.043200854 0
0.0 1 21 192.774256646 246.406420338 0
0.0 1 22 201.88348472 144.289212315 0
0.0 1 23 229.824788855 169.097144847 0
0.0 1 24 234.431511826 247.607613337 0
0.0 1 25 260.737628449 212.011968093 0
0.0 

80.0 0 3 218.117911117 234.457254338 0
80.0 1 4 157.186362067 207.865030081 0
80.0 1 5 162.744247573 235.781815279 0
80.0 1 6 170.392557553 181.106535479 0
80.0 1 7 177.395269286 197.280212411 0
80.0 1 8 167.407662623 164.722456649 0
80.0 1 9 169.200150017 206.088133207 0
80.0 1 10 231.56910896 200.894584721 0
80.0 0 11 176.393494592 146.69321657 0
80.0 1 12 206.074985931 226.691403026 0
80.0 1 13 222.472731951 208.989197715 0
80.0 0 14 192.002261815 254.611371742 0
80.0 1 15 213.72952444 159.599452595 0
80.0 0 16 142.042554776 219.855403039 0
80.0 0 17 146.316698908 183.015728546 0
80.0 0 18 233.147610509 159.794816225 0
80.0 1 19 197.711529011 218.889761428 0
80.0 0 20 217.552065585 250.574761552 0
80.0 1 21 193.310238744 239.896963894 0
80.0 1 22 198.048008689 158.714995307 0
80.0 1 23 226.281835165 171.619507839 0
80.0 0 24 233.601553702 245.736783914 0
80.0 0 25 260.922555558 227.19378049 0
80.0 1 26 202.426284257 172.821830599 0
80.0 0 27 183.283640674 172.183945693 0
80.0 0 28 1

160.0 1 9 172.750440018 204.070396595 0
160.0 1 10 226.227909073 196.258262665 0
160.0 0 11 179.853924014 163.249929552 1
160.0 1 12 198.826886748 236.6106658 0
160.0 1 13 214.855989156 210.250425491 0
160.0 0 14 185.088954024 243.704359451 1
160.0 1 15 207.568632324 173.856377117 0
160.0 0 16 153.530250948 217.295479592 1
160.0 0 17 157.430334064 185.170598845 1
160.0 0 18 221.967796764 168.320876996 1
160.0 1 19 192.749786216 214.434290106 0
160.0 0 20 216.985612979 243.216343878 1
160.0 0 21 195.001921015 246.741649905 1
160.0 1 22 196.637301306 170.082036809 0
160.0 1 23 217.348815476 178.592649343 0
160.0 0 24 228.055084738 239.020206521 1
160.0 0 25 236.199779383 218.710638378 1
160.0 1 26 195.687239707 181.724683831 0
160.0 1 27 190.675090134 192.352336499 1
160.0 0 28 163.551377896 230.729737201 1
160.0 1 29 206.94299228 184.503716271 0
160.0 1 30 213.305634893 221.825092235 0
160.0 0 31 162.137368525 177.008588369 1
160.0 0 32 190.932808636 160.174993501 1
160.0 1 33 167.81713

240.0 1 6 184.177638771 175.924606504 0
240.0 1 7 180.913221779 193.329518249 0
240.0 0 8 158.178173066 165.797669151 0
240.0 1 9 170.524699392 198.634401471 0
240.0 1 10 235.389873688 183.137269782 0
240.0 0 11 175.852881138 160.04177495 0
240.0 1 12 201.800051945 239.281305429 0
240.0 1 13 222.260692741 210.559915087 0
240.0 0 14 176.060575312 249.857216307 0
240.0 1 15 211.907185755 164.187984272 0
240.0 0 16 141.766662456 225.089623337 0
240.0 0 17 148.913771523 180.167038924 0
240.0 0 18 222.982141634 158.044920618 0
240.0 1 19 190.870053913 219.22780953 0
240.0 0 20 221.956233976 249.913084855 0
240.0 0 21 190.350277717 247.983693717 0
240.0 1 22 198.027066074 154.061003 0
240.0 1 23 224.962954559 171.595673074 0
240.0 1 24 226.165470598 238.632636218 0
240.0 0 25 249.045504208 224.248329188 0
240.0 1 26 199.595931515 172.5374944 0
240.0 1 27 191.015918495 187.039149043 1
240.0 0 28 153.735361805 235.494666285 0
240.0 1 29 209.519931826 179.818263863 0
240.0 1 30 218.045669863 22

320.0 1 9 164.63200967 198.700450987 0
320.0 1 10 226.33417027 191.874055946 0
320.0 0 11 166.915266005 163.830693178 1
320.0 1 12 208.9427236 238.102956175 0
320.0 1 13 216.954057426 213.82129739 0
320.0 0 14 180.85902656 246.59943174 0
320.0 0 15 210.868372193 158.041659696 1
320.0 0 16 126.784946363 235.793973071 0
320.0 1 17 161.855801557 183.867652671 1
320.0 0 18 220.101564492 164.072226034 1
320.0 1 19 186.144958429 221.26418164 0
320.0 0 20 222.815071185 263.623869463 0
320.0 1 21 200.02039906 246.23621461 0
320.0 0 22 189.172001367 152.332509143 1
320.0 1 23 221.34291324 181.557424504 0
320.0 0 24 224.851655157 245.281575048 0
320.0 0 25 249.699557367 230.260412335 0
320.0 1 26 198.081384738 165.597092197 0
320.0 1 27 198.976233769 177.832119667 1
320.0 0 28 150.542865836 236.020856446 0
320.0 1 29 210.667956115 171.83757977 0
320.0 1 30 221.872422043 227.541560548 0
320.0 1 31 172.675765494 188.08212234 0
320.0 0 32 176.547206362 159.177040379 1
320.0 1 33 188.077160478 183.0

400.0 0 8 175.086347226 164.775989248 1
400.0 0 9 158.908507407 173.078926886 1
400.0 0 10 233.462617338 189.061749074 1
400.0 0 11 183.827588002 161.025710941 1
400.0 1 12 212.148774186 236.543984002 0
400.0 1 13 224.635825114 209.30715327 0
400.0 0 14 179.348719832 246.759207143 0
400.0 0 15 204.835082169 163.093734817 1
400.0 0 16 122.612077661 231.61264257 0
400.0 1 17 168.682657155 192.7813592 1
400.0 1 18 203.949180784 175.063673855 1
400.0 1 19 187.361207204 227.797972125 0
400.0 0 20 219.099703964 253.757747413 0
400.0 0 21 195.761911655 252.54975054 0
400.0 1 22 194.746353405 169.43514865 1
400.0 1 23 212.726715032 198.913606432 0
400.0 0 24 232.766744823 247.853067256 0
400.0 0 25 249.200205874 219.597832618 0
400.0 1 26 195.609901463 193.719084555 0
400.0 1 27 200.576535025 204.758555394 1
400.0 0 28 135.657559268 234.998273157 0
400.0 1 29 206.191696855 187.028545715 0
400.0 1 30 224.93761946 239.175050636 0
400.0 0 31 157.596622512 191.524314578 1
400.0 1 32 184.702645482 

480.0 1 6 176.558224845 197.654401292 0
480.0 1 7 167.671683658 213.272408537 0
480.0 1 8 180.48400324 166.645288011 0
480.0 0 9 156.572824236 172.652317239 0
480.0 0 10 241.932123892 193.157004599 0
480.0 0 11 183.049207427 153.95049489 0
480.0 1 12 206.164965496 246.347286701 0
480.0 1 13 225.757602956 208.918066741 0
480.0 0 14 183.336644355 253.972748189 0
480.0 0 15 209.201024525 157.150964617 0
480.0 0 16 129.326431797 218.244283244 0
480.0 1 17 163.738934066 197.363778472 1
480.0 1 18 210.613031482 177.292604347 1
480.0 1 19 183.628534477 230.535381806 0
480.0 0 20 229.781283963 257.066398511 0
480.0 0 21 194.213108295 257.066145391 0
480.0 1 22 195.915664294 153.13496346 1
480.0 1 23 213.765703654 202.886839085 0
480.0 0 24 244.100152079 243.297196631 0
480.0 0 25 257.625952125 216.966146023 0
480.0 1 26 192.912803454 196.236997318 0
480.0 1 27 201.911226341 205.246186876 1
480.0 0 28 139.477096355 224.091826439 0
480.0 1 29 205.658586816 189.851827361 0
480.0 1 30 230.15989242